In [1]:
import duckdb  # Importa o DuckDB para manipulação de dados e execução de SQL
import os  # Importa o módulo os para interagir com variáveis de ambiente do sistema
from dotenv import load_dotenv, find_dotenv  # Importa funções para carregar variáveis de ambiente de um arquivo .env

from deltalake import DeltaTable, write_deltalake
# Carrega as variáveis de ambiente definidas no arquivo .env
load_dotenv(find_dotenv())

# Define as credenciais de acesso ao MinIO a partir das variáveis de ambiente
AWS_ACCESS_KEY = os.getenv("AWS_ACCESS_KEY_MINIO")  # Chave de acesso do MinIO
AWS_SECRET_KEY = os.getenv("AWS_SECRET_KEY_MINIO")  # Chave secreta do MinIO
HOST_MINIO = os.getenv("HOST_MINIO")  # Host do MinIO

# Conecta ao DuckDB, criando uma instância de conexão
con = duckdb.connect()

# Cria uma secret no DuckDB para acessar o MinIO com as credenciais fornecidas
con.execute(f"""
    CREATE SECRET my_minio_secret (
        TYPE 'S3',
        KEY_ID '{AWS_ACCESS_KEY}',
        SECRET '{AWS_SECRET_KEY}',
        REGION 'us-east-1',
        ENDPOINT '{HOST_MINIO}:9000',
        URL_STYLE 'path',
        USE_SSL false
    );
""")

# Define o caminho de destino no MinIO para os arquivos Parquet
path_minio_landing = 's3://landing/comex'
path_minio_bronze = 's3://bronze/comex/pais'

In [2]:
storage_options = {
    "AWS_ENDPOINT_URL": f"http://{HOST_MINIO}:9000",
    "AWS_REGION": "us-east-1",
    "AWS_ACCESS_KEY_ID": AWS_ACCESS_KEY,
    "AWS_SECRET_ACCESS_KEY": AWS_SECRET_KEY,
    "AWS_S3_ALLOW_UNSAFE_RENAME": "true",
    "AWS_ALLOW_HTTP": "true"
}

In [3]:
df = con.sql(f"""
        SELECT 
        CAST(CO_PAIS as int) AS cod_pais,
        CO_PAIS_ISOA3 AS cod_pais_iso,
        NO_PAIS AS nome_pais
         from '{path_minio_landing}/PAIS.parquet';        
        """).to_arrow_table()

In [4]:
df

pyarrow.Table
cod_pais: int32
cod_pais_iso: string
nome_pais: string
----
cod_pais: [[0,13,15,17,20,...,994,995,997,998,999]]
cod_pais_iso: [["ZZZ","AFG","ALA","ALB","ESP",...,"ZZZ","ZZZ","ZZZ","ZZZ","ZZZ"]]
nome_pais: [["Não Definido","Afeganistão","Aland, Ilhas","Albânia","Alboran-Perejil, Ilhas",...,"A Designar","Bancos Centrais","Organizações Internacionais","Sem informação","Não Declarados"]]

In [7]:
write_deltalake(
        f'{path_minio_bronze}',
        df,
        storage_options=storage_options,
        
    )

In [5]:
table_path = f'{path_minio_bronze}'

# Conecte à tabela Delta existente
table = DeltaTable(table_path, storage_options=storage_options)



In [6]:
(
table.merge(
    source=df,
    predicate='target.cod_pais = source.cod_pais',
    source_alias="source",
    target_alias="target",
).when_not_matched_insert_all().execute() 

)

{'num_source_rows': 281,
 'num_target_rows_inserted': 139,
 'num_target_rows_updated': 0,
 'num_target_rows_deleted': 0,
 'num_target_rows_copied': 0,
 'num_output_rows': 139,
 'num_target_files_added': 1,
 'num_target_files_removed': 0,
 'execution_time_ms': 85,
 'scan_time_ms': 0,
 'rewrite_time_ms': 44}

In [7]:
con.close()